# Crop yield prediction using perceptron model


In [7]:
import numpy as np
import pandas as pd
import csv

In [8]:
data = None

In [9]:
with open('data/bagalkot.csv', 'r') as data_file:
    data = pd.read_csv(data_file, delimiter=',')
data.head()

,ï»¿Crop,Year,Rainfall,Temp,N,P,K,Yield(Tons/Hectre)
0,Bajra,1998,815.087,26.0,39.11,41.25,7.13,1.07
1,Bajra,1999,768.995,27.0,39.11,41.25,7.13,0.73
2,Bajra,2000,857.558,28.0,39.11,41.25,7.13,1.00
3,Bajra,2001,607.528,25.0,39.11,41.25,7.13,0.76
4,Bajra,2002,632.970,27.0,39.11,41.25,7.13,0.30


## Training data
- X_train :- N,P,K,Rainfall and temperature data
- Y-train :- Yield data

In [10]:
X_train = []
Y_train = []

In [11]:
for _, row in data.iterrows():
    X_train.append([row["Rainfall"], row["Temp"], row["N"],row["P"], row["K"]])
    Y_train.append(row["Yield(Tons/Hectre)"])

X_train = np.array(X_train)
Y_train = np.array(Y_train)*1000
print(X_train.shape)
print(Y_train.shape)

(180, 5)
(180,)


## Yield prediction done according to following points
- Prediction formula is Wi*Xi+bias where W= Weight of X , X = (N,P,K,Rainfall,Temp).
- Network is of 5 nodes with N,P,K,Rainfall,Temp

In [13]:
weights = None
bias = None
#step value
def prediction(X, W, b):

    return ((np.matmul(X,W)+b)[0])

#functon for getting weight
def Stepfunction(X, y, W, b, learn_rate = 0.01):

    for i in range(0, len(X)):

        y_hat = prediction(X[i], W, b)

        if y[i] > y_hat:

            W[0] += (X[i][0]*learn_rate)
            W[1] += (X[i][1]*learn_rate)
            W[2] += (X[i][2]*learn_rate)
            W[3] += (X[i][3]*learn_rate)
            W[4] += (X[i][4]*learn_rate)

            
            b += learn_rate

        elif y[i] < y_hat:

            W[0] -= (X[i][0]*learn_rate)
            W[1] -= (X[i][1]*learn_rate)
            W[2] -= (X[i][2]*learn_rate)
            W[3] -= (X[i][3]*learn_rate)
            W[4] -= (X[i][4]*learn_rate)
            
            b -= learn_rate

    return W, b

#function to find the equation of boundary lines
def trainmodel(X, y, learn_rate=0.00005, num_epoches=10000):

    global weights, bias
    #finding max and min values
    x1_min, x1_max = min(X.T[0]), max(X.T[0])
    x2_min, x2_max = min(X.T[1]), max(X.T[1])
    x3_min, x3_max = min(X.T[2]), max(X.T[2])
    x4_min, x4_max = min(X.T[3]), max(X.T[3])
    x5_min, x5_max = min(X.T[4]), max(X.T[4])
    #x6_min, x6_max = min(X.T[5]), max(X.T[5])
    #x7_min, x7_max = min(X.T[6]), max(X.T[6])
    
    W = np.array(np.random.rand(5,1)) 

    b = np.random.rand(1)[0] + (x1_max+x2_max+x3_max+x4_max+x5_max)/5#+x6_max+x7_max)/7

    #training the model
    for i in range(0,num_epoches):
        W, b = Stepfunction(X, y, W, b, learn_rate)
    weights = W
    bias = b
    print(W,b)

if __name__ == '__main__':
    trainmodel(X_train[:20], Y_train[:20])
    idx =13
    pred=np.matmul(X_train[idx],weights)+bias
    act=Y_train[idx]
    print("Predicted :",pred)
    print("Actual :",act)
    error = abs(pred - act)
    print("Error: ", error)
    print('Mean Absolute Error:', round(np.mean(error), 3), 'degrees.')
    mape = 100 * (error / act)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 4), '%')

[[ 1.0685562 ]
 [ 0.33523849]
 [-0.62530874]
 [-1.1990951 ]
 [-0.26874294]] 244.55925243512308
Predicted : [1330.36539324]
Actual : 1280.0
Error:  [50.36539324]
Mean Absolute Error: 50.365 degrees.
Accuracy: 96.0652 %


## Saving weights of all nodes in a model

In [14]:
def save_model(array, name):
    np.savetxt(name+".csv", array, delimiter=",")

In [15]:
def load_model(name):
    with open(name+".csv", 'r') as read_file:
        reader = csv.reader(read_file)
        data = []
        for row in reader:
            data.append(row[0])
    read_file.close()
    data = np.asarray(data, dtype=np.float64)
    return data

## Save model for every crop in every district of karnataka

In [16]:
wb = np.append(weights, bias)
#wb.append(bias)
print(wb)
save_model(wb, 'bagalkot_wheat')

[  1.0685562    0.33523849  -0.62530874  -1.1990951   -0.26874294
 244.55925244]
